# Analyzing Campaign Finance Data to Predict Election Results

**Tony Wu  
CMSC 320  
Fall 2022**

## Introduction

In [48]:
from zipfile import ZipFile
from fuzzywuzzy import fuzz, process
from itertools import product
import json
import pandas as pd
import requests

In [2]:
finance_url = "https://www.fec.gov/files/bulk-downloads/2022/weball22.zip"
house_url = "https://bigbuilder.s3.us-west-1.amazonaws.com/projects/elections-data/2022-11-08/latest-house.json"
senate_url = "https://bigbuilder.s3.us-west-1.amazonaws.com/projects/elections-data/2022-11-08/latest-senate.json"

## Data Collection

In [3]:
response = requests.get(finance_url)
with open("candidates.zip", "wb") as f:
    f.write(response.content)

# Unzip into data directory
with ZipFile("candidates.zip", "r") as f:
    f.extractall()

URL: https://www.latimes.com/projects/2022-election-US-house-senate-governors-live-results
Since the website loads the election table dynamically, we can't scrape easily scrape it. So, we can look in the networks tab to see where the data is being loaded in from.

In [4]:
response = requests.get(house_url)
with open("house.json", "wb") as f:
    f.write(response.content)

response = requests.get(senate_url)
with open("senate.json", "wb") as f:
    f.write(response.content)

In [80]:
finance_df = pd.read_csv("weball22.txt", sep="|", header=None)[[1, 4, 5, 7, 17, 18]]
finance_df.columns = ["name", "party", "total_contribution", "total_spent", "individual_contributions", "state"]
finance_df

df = finance_df["name"].str.split(", ", n=1, expand=True)
df.columns = ["last_name", "first_name"]
finance_df = df.join(finance_df.iloc[:, 1:])
finance_df

,last_name,first_name,party,total_contribution,total_spent,individual_contributions,state
0,CONSTANT,CHRISTOPHER,DEM,164637.90,164037.51,158023.05,AK
1,PELTOLA,MARY,DEM,7624367.30,6506330.37,7061074.35,AK
2,WOOL,ADAM L,DEM,16217.07,16217.07,15117.00,AK
3,REVAK,JOSHUA CARL,REP,121841.00,121841.00,116666.00,AK
4,PALIN,SARAH,REP,1967904.40,1908104.64,1768597.87,AK
...,...,...,...,...,...,...,...
4125,BEN DAVID,MERAV,DEM,0.00,14278.00,0.00,WY
4126,LUMMIS,CYNTHIA MARIE MRS.,REP,367850.30,352180.98,170844.90,WY
4127,MILLER,BRYAN,REP,0.00,0.00,0.00,WY
4128,BARRASSO,JOHN A,REP,1483119.58,1323311.92,943965.09,WY


In [81]:
finance_df = (
    finance_df[finance_df["total_spent"] >= 0]
    .sort_values("total_spent", ascending=False)
    .drop_duplicates(subset=["first_name", "last_name"], keep="first")
)
finance_df

,last_name,first_name,party,total_contribution,total_spent,individual_contributions,state
3376,MERCER JR,LEE,DEM,3.840000e+08,3.840000e+08,0.000000e+00,00
3616,WARNOCK,RAPHAEL,DEM,1.756922e+08,1.687255e+08,1.645636e+08,GA
3507,KELLY,MARK,DEM,9.235173e+07,9.165264e+07,8.577113e+07,AZ
681,DEMINGS,VALDEZ 'VAL',DEM,8.092734e+07,8.106547e+07,7.858148e+07,FL
3589,DEMINGS,VAL,DEM,8.092734e+07,8.106547e+07,7.858148e+07,FL
...,...,...,...,...,...,...,...
3595,LAROSE,JOSUE DR.,DEM,0.000000e+00,0.000000e+00,0.000000e+00,FL
1061,CAMPBELL,MATTHEW,DEM,0.000000e+00,0.000000e+00,0.000000e+00,IA
334,KLEINMAN,PABLO,REP,0.000000e+00,0.000000e+00,0.000000e+00,CA
1063,JONES,NICHOLAS,REP,0.000000e+00,0.000000e+00,0.000000e+00,ID


In [102]:
data = json.load(open("house.json")) + json.load(open("senate.json"))
candidates = []

# Runoff elections
# 2015 -> 3156
# 20645 -> 21405
# 2933 -> 3153

for row in data:
    # Remove duplicate candidate data and duplicate runoff elections
    if row["id"] in ["15766", "21404", "3156", "21405", "3153", "8516"]:
        continue

    for i, candidate in enumerate(row["candidates"]):
        # Remove candidates with no votes and write-in candidates
        if candidate["votes"] == 0 or candidate["family_name"] == "Total Write-Ins":
            continue

        # For the runoff elections, the first candidate won each time
        if row["id"] in ["2015", "20645", "2933"] and i == 0:
            candidate["is_winner"] = True

        # District is 0 for senate
        if row["office_id"] == "S":
            district = 0
        else:
            district = row["seat_number"]

        if candidate["party"] == "gop":
            candidate["party"] = "REP"

        candidates.append(
            {
                "last_name": candidate["family_name"].upper(),
                "first_name": candidate["given_name"].upper(),
                "state": row["state_postal"],
                "district": district,
                "party": candidate["party"].upper(),
                "is_winner": candidate["is_winner"],
                "is_incumbent": candidate["is_incumbent"],
                "votes": candidate["votes"],
            }
        )

votes_df = pd.DataFrame(candidates)
votes_df

,last_name,first_name,state,district,party,is_winner,is_incumbent,votes
0,PELTOLA,MARY,AK,1,DEM,True,True,128329
1,PALIN,SARAH,AK,1,REP,False,False,67732
2,BEGICH,NICK,AK,1,REP,False,False,61431
3,BYE,CHRIS,AK,1,LIB,False,False,4560
4,CARL,JERRY,AL,1,REP,True,True,139854
...,...,...,...,...,...,...,...,...
1165,ERICSON,MS. CRIS,VT,0,IND,False,False,1102
1166,MURRAY,PATTY,WA,0,DEM,True,True,1741827
1167,SMILEY,TIFFANY,WA,0,REP,False,False,1299322
1168,JOHNSON,RON,WI,0,REP,True,True,1336928


In [107]:
data = []

for _, row in votes_df.iterrows():
    df = finance_df[
        (
            (finance_df["last_name"].str.contains(row["last_name"]))
            & (finance_df["first_name"].str.contains(row["first_name"]))
        )
    ]

    if len(df) == 0:
        name = row["first_name"] + " " + row["last_name"]
        state_df = finance_df[
            (finance_df["state"] == row["state"])
            & (finance_df["party"] == row["party"])
            & (
                (finance_df["first_name"] == row["first_name"])
                | (finance_df["last_name"] == row["last_name"])
            )
        ]
        if len(state_df) > 0:
            closest = process.extract(
                name, state_df["first_name"] + " " + state_df["last_name"], limit=1
            )[0]
            if closest[1] >= 80:
                df = finance_df[
                    finance_df["first_name"] + " " + finance_df["last_name"]
                    == closest[0]
                ]

    if len(df) >= 1:
        finance_row = df.loc[df["total_spent"].idxmax()]
        data.append(
            {
                "last_name": row["last_name"],
                "first_name": row["first_name"],
                "state": row["state"],
                "district": row["district"],
                "party": row["party"],
                "is_winner": row["is_winner"],
                "is_incumbent": row["is_incumbent"],
                "votes": row["votes"],
                "total_spent": finance_row["total_spent"],
            }
        )

data_df = pd.DataFrame(data)
data_df

,last_name,first_name,state,district,party,is_winner,is_incumbent,votes,total_spent
0,PELTOLA,MARY,AK,1,DEM,True,True,128329,6506330.37
1,PALIN,SARAH,AK,1,REP,False,False,67732,1908104.64
2,BEGICH,NICK,AK,1,REP,False,False,61431,1552558.33
3,BYE,CHRIS,AK,1,LIB,False,False,4560,3940.25
4,CARL,JERRY,AL,1,REP,True,True,139854,931386.18
...,...,...,...,...,...,...,...,...,...
906,COESTER,MARK,VT,0,IND,False,False,1270,9394.38
907,MURRAY,PATTY,WA,0,DEM,True,True,1741827,18778781.04
908,SMILEY,TIFFANY,WA,0,REP,False,False,1299322,20150770.46
909,JOHNSON,RON,WI,0,REP,True,True,1336928,33830890.07


## Exploration

## Analysis

## Hypothesis Testing

## Conclusion